In [21]:
import pandas as pd
import numpy as np
import openrouteservice as ors
from dotenv import load_dotenv
from pyonemap import OneMap
import os
import requests
import json
import time
from collections import namedtuple
import helper_functions as helper
from datetime import datetime,timedelta

In [22]:
load_dotenv()

True

In [3]:
directory = os.getcwd()

os.chdir(directory)

In [4]:
hdbCentroids_df = pd.read_csv(r"../../data/Cluster_data/hdb_cluster_centroids.csv",header = None,names = ['Latitude','Longitude'])
priv_centroid_df = pd.read_csv(r"../../data/Cluster_data/priv_cluster_centroids.csv",header = None,names = ['Latitude','Longitude'])
mrt_stations_df = pd.read_csv(r"../../data/Cluster_data/mrt_station_final.csv",usecols = [1,2,3])

In [5]:
hdbCentroids_df.info()
priv_centroid_df.info()
mrt_stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Latitude   550 non-null    float64
 1   Longitude  550 non-null    float64
dtypes: float64(2)
memory usage: 8.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Latitude   550 non-null    float64
 1   Longitude  550 non-null    float64
dtypes: float64(2)
memory usage: 8.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MRT.Name   175 non-null    object 
 1   Latitude   175 non-null    float64
 2   Longitude  175 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.2+ KB


In [6]:
mrt_stations_df.sort_values(by='MRT.Name', inplace=True)

mrt_stations_df.head(1)

,MRT.Name,Latitude,Longitude
12,ADMIRALTY MRT STATION,1.440589,103.80099


In [7]:
combined_centroid_df = pd.concat([hdbCentroids_df,priv_centroid_df],axis = 0).reset_index(drop = True)

In [8]:
combined_centroid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Latitude   1100 non-null   float64
 1   Longitude  1100 non-null   float64
dtypes: float64(2)
memory usage: 17.3 KB


In [9]:
#create a psuedo index for my residential centroids df
combined_centroid_df['index'] = combined_centroid_df.index

combined_centroid_df['index']

#create a psuedo index for my residential centroids df
mrt_stations_df['index'] = mrt_stations_df.index

mrt_stations_df['index']

##create a dummy variable to cross join on 
combined_centroid_df['join_key'] = "A"
mrt_stations_df['join_key'] = "A"

#Cross join to obtain combinations of all possible pairings between MRTs and Residential Centroids
centroid_station_combination_df = pd.merge(combined_centroid_df, mrt_stations_df, on='join_key')

In [11]:
coordinate_pair = namedtuple('coordinate_pair',['lat_x','lon_x','lat_y','lon_y'])

In [19]:
#Apply function to dataframe and store distances in new column 'euclidean distance'
centroid_station_combination_df['euclidean_distance'] = helper.haversine(centroid_station_combination_df['Latitude_x'], centroid_station_combination_df['Longitude_x'], centroid_station_combination_df['Latitude_y'], centroid_station_combination_df['Longitude_y'])
#Group by residential centroid, filter out the closest MRT by distance for each centroid into another dataframe and reset its index
result_df = centroid_station_combination_df.groupby('MRT.Name').apply(lambda group: group.nsmallest(5, 'euclidean_distance')).reset_index(drop=True)
#result_df['MRT.Name'] = pd.merge(result_df, mrt_stations_df, left_on='index_y', right_on='index')['MRT.Name']
#Create a new column 'coordinate_pair' to store coordinate pairs to pass to openrouteservice API direction query
result_df['coordinate_pair'] = result_df.apply(lambda x: coordinate_pair(x['Latitude_x'], x['Longitude_x'], x['Latitude_y'], x['Longitude_y']), axis=1)
#create an empty column 'route' to later store query response
result_df['cycle_route'] = np.nan
result_df['bus_route'] = np.nan

KeyError: 'MRT.Name'

In [18]:
result_df.info()

counts = result_df.groupby('MRT.Name').size().reset_index(name='Count')

counts[counts['Count'] != 5]  # sanity check

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Latitude_x          875 non-null    float64
 1   Longitude_x         875 non-null    float64
 2   index_x             875 non-null    int64  
 3   join_key            875 non-null    object 
 4   MRT.Name            875 non-null    object 
 5   Latitude_y          875 non-null    float64
 6   Longitude_y         875 non-null    float64
 7   index_y             875 non-null    int64  
 8   euclidean_distance  875 non-null    float64
 9   coordinate_pair     875 non-null    object 
 10  cycle_route         0 non-null      float64
 11  bus_route           0 non-null      float64
dtypes: float64(7), int64(2), object(3)
memory usage: 82.2+ KB


,MRT.Name,Count


In [23]:
one_map_email = os.getenv("ONE_MAP_EMAIL")
one_map_password = os.getenv("ONE_MAP_PASSWORD")
payload = {
        "email": one_map_email,
        "password": one_map_password
      }
api_key = requests.request("POST", "https://www.onemap.gov.sg/api/auth/post/getToken", json=payload)
api_key = api_key.json()["access_token"]

onemap = OneMap(api_key)

In [24]:
result_df['cycle_route'] = result_df['coordinate_pair'].apply(lambda x:helper.get_cycle_route(onemap,x))

In [25]:
result_df['distance'] = result_df['cycle_route'].apply(helper.get_distance)
result_df['cycle_duration'] = result_df['cycle_route'].apply(helper.get_time)

In [29]:
#result_df['centroid_name'] = result_df.apply(helper.get_centroid_name, axis=1)
result_df['centroid_name'] = result_df.apply(lambda row: helper.get_centroid_name(onemap, row),axis=1)

In [30]:
ors_key = os.getenv("ORS_API_KEY")

client = ors.Client(key= ors_key)

In [31]:
result_df['suitability'] = result_df['cycle_route'].apply(lambda x: helper.get_path_suitability(client,x))

/Users/ethan/Library/Python/3.11/lib/python/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/ethan/Library/Python/3.11/lib/python/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/ethan/Library/Python/3.11/lib/python/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/ethan/Library/Python/3.11/lib/python/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/ethan/Lib

In [32]:
result_df['steepness'] = result_df['cycle_route'].apply(lambda x: helper.get_path_steepness(client,x))

/Users/ethan/Library/Python/3.11/lib/python/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/ethan/Library/Python/3.11/lib/python/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/ethan/Library/Python/3.11/lib/python/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/ethan/Library/Python/3.11/lib/python/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/ethan/Lib

In [33]:
#result_df[['bus_route', 'bus_duration']] = result_df.apply(lambda row: pd.Series(helper.apply_fetch_itinerary(row)), axis=1)
#result_df['time_difference'] = result_df['cycle_duration'] - result_df['bus_duration']

new_result_df = result_df.loc[:, result_df.columns != 'route']
new_result_df.to_csv(r"../../data/Cluster_data/five_point_Centroid_MRT pairing data.csv")


routes = result_df['cycle_route'].copy(deep = True)
routes.to_json(r'../../data/Cluster_data/five_point_Centroid_MRT_cycle_routes.json', orient='records')

#result_df.to_csv(r"..\data\five_point_Centroid_MRT pairing data.csv")


computing bus route timings

In [35]:
df = pd.read_csv("../../data/Cluster_data/five_point_Centroid_MRT pairing data.csv", index_col=0)
df.columns

Index(['Latitude_x', 'Longitude_x', 'index_x', 'join_key', 'MRT.Name',
       'Latitude_y', 'Longitude_y', 'index_y', 'euclidean_distance',
       'coordinate_pair', 'cycle_route', 'bus_route', 'distance',
       'cycle_duration', 'centroid_name', 'suitability', 'steepness'],
      dtype='object')

In [36]:
import requests
import json

def fetch_itinerary(from_lat, from_lon, to_lat, to_lon):
    # Define the GraphQL endpoint
    endpoint = 'http://localhost:8080/otp/gtfs/v1'
    
    # Construct the GraphQL query
    query = """
    query EgQuery($time: String, $date: String, $from: InputCoordinates, $to: InputCoordinates, $numItineraries: Int, $walkReluctance: Float) {
      plan(
        time: $time
        date: $date
        from: $from
        to: $to
        transportModes: [
            {
                mode: WALK
            },
            {
                mode: TRANSIT
            },
        ]
        numItineraries: $numItineraries
        walkReluctance: $walkReluctance
      ) {
        itineraries {
          legs {
                mode
                startTime
                endTime
                from {
                    name
                    lat
                    lon
                    departureTime
                    arrivalTime
                }
                to {
                    name
                    lat
                    lon
                    departureTime
                    arrivalTime
                }
                route {
                    gtfsId
                    longName
                    shortName
                }
                legGeometry {
                    points
                }
            }
          duration
        }
      }
    }
    """

    # Define the variables for your query
    variables = {
        'time': '18:00:00',
        'date': '2023-06-01',
        'from': {'lat': from_lat, 'lon': from_lon},
        'to': {'lat': to_lat, 'lon': to_lon},
        'numItineraries': 1,
        'walkReluctance': 3.5
    }

    # Set up the headers
    headers = {
        'Content-Type': 'application/json'
    }

    # Send the request to the GraphQL API
    response = requests.post(endpoint, headers=headers, json={'query': query, 'variables': variables})

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"GraphQL query failed to run with a {response.status_code}")



In [37]:
def apply_fetch_itinerary(row):
    try:
        response = fetch_itinerary(
            from_lat=row['Latitude_x'], 
            from_lon=row['Longitude_x'], 
            to_lat=row['Latitude_y'], 
            to_lon=row['Longitude_y']
        )
        bus_route = []
        # Extract relevant information from response
        if response and 'data' in response and 'plan' in response['data'] and 'itineraries' in response['data']['plan']:
            itinerary = response['data']['plan']['itineraries'][0]  # Taking the first itinerary
            duration = itinerary['duration'] if 'duration' in itinerary else None
            for leg in itinerary['legs']:
                # if leg['mode'] == 'BUS':
                    # Append bus route information for each bus leg
                from_stop = leg['from']['name']
                to_stop = leg['to']['name']
                bus_route.append(f"{from_stop} to {to_stop}")
                #bus_route.append(leg['route']['shortName'])
                #bus_route.append(f"{leg['legGeometry']['points']}")
            # Further extraction can be done based on the structure of your response
            return bus_route, duration/60  # Convert seconds to minutes
        else:
            return None, None
    except Exception as e:
        print(f"Error fetching itinerary: {e}")
        return None, None

# Apply the function to each row of the DataFrame
df[['bus_route', 'bus_duration']] = df.apply(lambda row: pd.Series(apply_fetch_itinerary(row)), axis=1)
df['time_difference'] = df['cycle_duration'] - df['bus_duration']


In [38]:
df.to_csv(r"../../data/Cluster_data/five_point_Centroid_MRT pairing data.csv.csv")